In [1]:
import sklearn
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn import metrics

from joblib import dump, load

### Loading the Yelp dataset

In [2]:
os.makedirs('/home/jupyter/yelp', exist_ok=True)
os.makedirs('/home/jupyter/data/yelp/yelp_model', exist_ok=True)

In [3]:
directory = 'https://storage.googleapis.com/msca-bdp-data-open/yelp/'
fileName = 'yelp_train_sentiment.json'

path = directory + fileName

In [4]:
%%time

yelp = pd.read_json(path, orient='records', lines=True)
yelp.shape

CPU times: user 1.67 s, sys: 705 ms, total: 2.37 s
Wall time: 2.74 s


(255717, 3)

In [5]:
pd.set_option('display.max_colwidth', 200)

In [6]:
# examine the first 5 rows
yelp.head(5)

,text,label,lang
0,"I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sa...",1,en
1,I love the classes at this gym. Zumba and. Radio Hip Hop are my favorite. This is such a great fun and I love that it is so reasonably priced!,1,en
2,The tables and floor were dirty. I was the only customer on a Saturday nite and the person working the counter ignored me I had a corned beef sandwich. I took three bites and threw it in the trash,0,en
3,"I had an oil change at the 15515 N Scottsdale Road location. When the car was delivered to me, there were two engine warning lights on that had not been on when I drove the car in. The technicia...",0,en
4,The absolute WORST apartment complex I have ever lived in. Moved here from out of state. Hoped to find a decently priced apartment until I got myself settled in. Wow this place has been trash. Lan...,0,en


In [7]:
# define X and y
X = yelp['text']
y = yelp['label']
print(X.shape)
print(y.shape)

(255717,)
(255717,)


In [8]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(191787,)
(63930,)
(191787,)
(63930,)


### Creating the Logistic Regression pipeline

In [9]:
pipe_logreg = make_pipeline(
    CountVectorizer(lowercase=False, stop_words='english', ngram_range=(1,3)),
    LogisticRegression(max_iter=1000)
)

In [10]:
%time pipe_logreg.fit(X_train, y_train)

CPU times: user 37min 29s, sys: 33min 3s, total: 1h 10min 33s
Wall time: 9min 57s


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(lowercase=False, ngram_range=(1, 3),
                                 stop_words='english')),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [11]:
%time y_pred = pipe_logreg.predict(X_test)

CPU times: user 11.9 s, sys: 16.2 ms, total: 11.9 s
Wall time: 11.9 s


In [12]:
print(f"Test Accuracy: {metrics.accuracy_score(y_test, y_pred) * 100:.1f}%")

Test Accuracy: 97.3%


In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     32016
           1       0.97      0.97      0.97     31914

    accuracy                           0.97     63930
   macro avg       0.97      0.97      0.97     63930
weighted avg       0.97      0.97      0.97     63930



In [14]:
%time dump(pipe_logreg, "/home/jupyter/data/yelp/yelp_model/logreg.joblib")

CPU times: user 1min 14s, sys: 2.02 s, total: 1min 16s
Wall time: 1min 16s


['/home/jupyter/data/yelp/yelp_model/logreg.joblib']

In [15]:
%time clf_large = load(os.path.join('/home/jupyter/data/yelp/yelp_model/', 'logreg.joblib'))

CPU times: user 39.1 s, sys: 1.77 s, total: 40.9 s
Wall time: 40.9 s


In [16]:
import multiprocessing

In [17]:
num_processors = multiprocessing.cpu_count()
num_processors

32

In [18]:
news_df = pd.read_parquet("news_topics.parquet")

In [19]:
news_df

,url,date,language,title,text,cleaned_text,topic
0,http://en.people.cn/n3/2021/0318/c90000-9830122.html,2021-03-18,en,Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online,\n\nArtificial intelligence improves parking efficiency in Chinese cities - People's Daily Online\n\nHome\nChina Politics\nForeign Affairs\nOpinions\nVideo: We Are China\nBusiness\nMilitary\nWorld...,Artificial intelligence improves parking efficiency in Chinese cities Peoples Daily OnlineHomeChina PoliticsForeign AffairsOpinionsVideo: We Are ChinaBusinessMilitaryWorldSocietyCultureTravelScie...,4
3,http://www.homeoffice.consumerelectronicsnet.com/strategy-analytics-71-of-smartphones-sold-globally-in-2021-will-be-ai-powered/,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net,\n\nStrategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electroni...,Strategy Analytics: 71 of Smartphones Sold Globally in 2021 will be AI Powered Consumer Electronics Net Skip to contentConsumer Electronics NetPrimary MenuConsumer Electronics NetSearch for: Home...,8
13,https://9to5mac.com/2021/03/25/apple-bought-more-ai-companies-than-anyone-else-between-2016-and-2020/,2021-03-26,en,Apple bought more AI companies than anyone else between 2016 and 2020 - 9to5Mac,\nApple bought more AI companies than anyone else between 2016 and 2020 - 9to5Mac\n \nSwitch site\n\nExclusives\nGuides\n\nReviews\nHow Tos\nAAPL\nApple Store\nApple Arcade\nApple Card\nApple Sili...,Apple bought more AI companies than anyone else between 2016 and 2020 9to5Mac Switch siteExclusivesGuidesReviewsHow TosAAPLApple StoreApple ArcadeApple CardApple SiliconApple OneCarPlaySiriHomeKi...,0
14,https://abcnews.go.com/Nightline/video/trump-deepfakes-social-media-prompt-warnings-ai-risks-98398905,2023-04-06,en,Video Trump deepfakes on social media prompt warnings of AI risks - ABC News,\n\nVideo Trump deepfakes on social media prompt warnings of AI risks - ABC News\nABC NewsVideoLiveShowsGuns in AmericaInterest Successfully AddedWe'll notify you here with news aboutTurn on deskt...,Video Trump deepfakes on social media prompt warnings of AI risks ABC NewsABC NewsVideoLiveShowsGuns in AmericaInterest Successfully AddedWell notify you here with news aboutTurn on desktop notif...,7
15,https://abcnews.go.com/Technology/video/eu-imposes-sweeping-regulations-facial-recognition-artificial-intelligence-69072589,2020-02-19,en,"EU imposes sweeping regulations on facial recognition, Artificial Intelligence Video - ABC News","\n\nEU imposes sweeping regulations on facial recognition, Artificial Intelligence Video - ABC News\n\nSections\nSections\n\nTop Stories\n\nVideo\n\nLive\n\nU.S.\n\nPolitics\n\n2020 Elections\n\n...","EU imposes sweeping regulations on facial recognition, Artificial Intelligence Video ABC NewsSectionsSectionsTop StoriesVideoLiveU.S.Politics2020 ElectionsInternationalEntertainmentBusinessTechn...",4
...,...,...,...,...,...,...,...
200324,https://www.wymt.com/prnewswire/2023/03/01/drift-introduces-gpt-integration-advancing-ai-b2b-marketers-sellers/,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI for B2B Marketers and Sellers","Drift Introduces GPT Integration, Advancing AI for B2B Marketers and Sellers\n\nSkip to contentWeatherNewsSportsSupport LocalWatch WYMT NowFlood ReliefHomeNewsFlood ReliefEducationInternationalNat...","Drift Introduces GPT Integration, Advancing AI for B2B Marketers and SellersSkip to contentWeatherNewsSportsSupport LocalWatch WYMT NowFlood ReliefHomeNewsFlood ReliefEducationInternationalNationa...",6
200325,https://www.wymt.com/prnewswire/2023/04/28/urologists-explore-ai-prostate-cancer-detection-effects-prostate-cancer-female-partners/,2023-04-28,en,Urologists Explore AI for Prostate Cancer Detection and the Effects of Prostate Cancer on Fem

In [20]:
content = news_df['cleaned_text']

In [21]:
%%time
# Make predictions
y_pred = clf_large.predict(content)
y_pred_prob = clf_large.predict_proba(content)

CPU times: user 9min 2s, sys: 1.65 s, total: 9min 3s
Wall time: 9min 3s


In [23]:
import numpy as np

In [24]:
# Convert predicted values to class labels
y_pred_class = np.where(y_pred < 1, 'Negative', 'Positive')

# Create new columns in news_df for positive probability and negative probability
news_df['Positive Probability'] = y_pred_prob[:, 1]  # Probability for positive class
news_df['Negative Probability'] = y_pred_prob[:, 0]  # Probability for negative class
news_df['Sentiment'] = y_pred_class

In [12]:
pd.set_option('display.max_colwidth', None)

In [28]:
news_df['Sentiment'].value_counts()

Negative    81003
Positive    45814
Name: Sentiment, dtype: int64

In [29]:
news_df.to_parquet('log_yelp_sentiment.parquet')

In [1]:
import pandas as pd
news_df = pd.read_parquet("log_yelp_sentiment.parquet")

In [2]:
news_df

,url,date,language,title,text,cleaned_text,topic,Positive Probability,Negative Probability,Sentiment
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,4,0.337937,0.662063,Negative
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...,8,0.347064,0.652936,Negative
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...,0,0.994807,0.005193,Positive
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...,7,0.000011,0.999989,Negative
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...,4,0.000050,0.999950,Negative
...,...,...,...,...,...,...,...,...,...,...
200324,https://www.wymt.com/prnewswire/2023/03/01/dri...,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...",6,0.000017,0.999983,Negative
200325,https://www.wymt.com/prnewswire/2023/04/28/uro...,2023-04-28,en,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,1,0.002482,0.997518,Negative
200326,https://www.yorkdispatch.com/story/opinion/con...,2023-04-16,en,Does AI mean the 4-day workweek is almost here?,Does AI mean the 4-day workweek is almost here...,Does AI mean the 4day workweek is almost hereT...,4,0.000038,0.999962,Negative
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...,5,1.000000,0.000000,Positive


In [3]:
news_df.drop('Sentiment', axis=1, inplace=True)

In [4]:
news_df

,url,date,language,title,text,cleaned_text,topic,Positive Probability,Negative Probability
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,4,0.337937,0.662063
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...,8,0.347064,0.652936
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...,0,0.994807,0.005193
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...,7,0.000011,0.999989
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...,4,0.000050,0.999950
...,...,...,...,...,...,...,...,...,...
200324,https://www.wymt.com/prnewswire/2023/03/01/dri...,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...",6,0.000017,0.999983
200325,https://www.wymt.com/prnewswire/2023/04/28/uro...,2023-04-28,en,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,1,0.002482,0.997518
200326,https://www.yorkdispatch.com/story/opinion/con...,2023-04-16,en,Does AI mean the 4-day workweek is almost here?,Does AI mean the 4-day workweek is almost here...,Does AI mean the 4day workweek is almost hereT...,4,0.000038,0.999962
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...,5,1.000000,0.000000


In [7]:
#Adjusting the threshold to avoid misclassification of sentiments due to the presence of strong words
news_df['Sentiment'] = news_df['Negative Probability'].apply(lambda x: 'Negative' if x > 0.85 else 'Positive')

In [8]:
news_df

,url,date,language,title,text,cleaned_text,topic,Positive Probability,Negative Probability,Sentiment
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,4,0.337937,0.662063,Positive
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...,8,0.347064,0.652936,Positive
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...,0,0.994807,0.005193,Positive
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...,7,0.000011,0.999989,Negative
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...,4,0.000050,0.999950,Negative
...,...,...,...,...,...,...,...,...,...,...
200324,https://www.wymt.com/prnewswire/2023/03/01/dri...,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...",6,0.000017,0.999983,Negative
200325,https://www.wymt.com/prnewswire/2023/04/28/uro...,2023-04-28,en,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,1,0.002482,0.997518,Negative
200326,https://www.yorkdispatch.com/story/opinion/con...,2023-04-16,en,Does AI mean the 4-day workweek is almost here?,Does AI mean the 4-day workweek is almost here...,Does AI mean the 4day workweek is almost hereT...,4,0.000038,0.999962,Negative
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...,5,1.000000,0.000000,Positive


In [9]:
news_df['Sentiment'].value_counts()

Negative    74045
Positive    52772
Name: Sentiment, dtype: int64

In [22]:
news_df[['title','cleaned_text','Negative Probability','Sentiment']]

title  \
0                    Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online   
3       Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net   
13                                 Apple bought more AI companies than anyone else between 2016 and 2020 - 9to5Mac   
14                                    Video Trump deepfakes on social media prompt warnings of AI risks - ABC News   
15                EU imposes sweeping regulations on facial recognition, Artificial Intelligence  Video - ABC News   
...                                                                                                            ...   
200324                                Drift Introduces GPT Integration, Advancing AI for B2B Marketers and Sellers   
200325   Urologists Explore AI for Prostate Cancer Detection and the Effects of Prostate Cancer on Female Partners   
200326                                                             Does AI mean the 4-day workweek is almost here?   
200329           From data to knowledge and AI via graphs: Technology to support a knowledge-based economy | ZDNet   
200330   [YS Learn] From helping diabetic patients to managing chronic illness: the journey of BeatO’s AI platform   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

### Saving the new data as a parquet file

In [23]:
news_df.to_parquet('yelp_sentiments.parquet')